# Predicting the S&P500 with Recurrent Neural Networks

### Outline
Recurrent neural networks are a special type of deep neural networks for learning outcomes where future predictions are linked to past events. The S&P 500 is a stock market index that tracks the performance of the 500 largest companies that are listed on stock exchanges in the United States. We may observe that the future price (in this case closing price) of an underlying (in this case, the S&P) may be linked to the past market data such as the previous high, low, opening and closing price of the underlying and the volume of trades made per day. Other information that affect the market in the past may also be contributing factors. In this project, we frame predicting the future closing price of the S&P 500 as a sequence problem. Hence, we use recurrent neural networks to build an approximation model for a multivariate time-series problem.

### Data source
Data is from YahooFinance (https://finance.yahoo.com/quote/%5EGSPC/history/).
90 Year Historical S&P 500 Index data since 1927. From December 30th 1927 to February 23rd 2021.
Historical data is inflation-adjusted using the headline CPI and each data point represents the month-end closing value. The current month is updated on an hourly basis with today's latest value. The current price of the S&P 500 as of February 23, 2021 is 3,881.37.